In [100]:
#importing libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
#Load in the data set
fifaDB = pd.read_csv("data.csv")

In [3]:
#Preview Data base
fifaDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
Unnamed: 0                  18207 non-null int64
ID                          18207 non-null int64
Name                        18207 non-null object
Age                         18207 non-null int64
Photo                       18207 non-null object
Nationality                 18207 non-null object
Flag                        18207 non-null object
Overall                     18207 non-null int64
Potential                   18207 non-null int64
Club                        17966 non-null object
Club Logo                   18207 non-null object
Value                       18207 non-null object
Wage                        18207 non-null object
Special                     18207 non-null int64
Preferred Foot              18159 non-null object
International Reputation    18159 non-null float64
Weak Foot                   18159 non-null float64
Skill Moves                 18159 non-null fl

In [4]:
fifaDB.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingT

In [5]:
#Drop unnecessary columns
Fifa = fifaDB.drop(['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Unnamed: 0', 'Photo', 'Flag', 'Club Logo', 'Joined', 'Loaned From', 'Contract Valid Until', 'Special', 'International Reputation', 'Real Face', "Nationality", "Age", "ID", "Release Clause", "Club","GKDiving", "GKHandling", "GKKicking", "GKPositioning", "GKReflexes" ], axis=1)

In [6]:
#Look for null values
Fifa.isna().sum() 

Name                0
Overall             0
Potential           0
Value               0
Wage                0
Preferred Foot     48
Weak Foot          48
Skill Moves        48
Work Rate          48
Body Type          48
Position           60
Jersey Number      60
Height             48
Weight             48
Crossing           48
Finishing          48
HeadingAccuracy    48
ShortPassing       48
Volleys            48
Dribbling          48
Curve              48
FKAccuracy         48
LongPassing        48
BallControl        48
Acceleration       48
SprintSpeed        48
Agility            48
Reactions          48
Balance            48
ShotPower          48
Jumping            48
Stamina            48
Strength           48
LongShots          48
Aggression         48
Interceptions      48
Positioning        48
Vision             48
Penalties          48
Composure          48
Marking            48
StandingTackle     48
SlidingTackle      48
dtype: int64

In [7]:
#Drop null values
Fifa = Fifa.dropna()

In [8]:
#Set index
Fifa.set_index('Name', inplace=True)

In [9]:
list_wages = Fifa['Wage']
wages = []
for i in list_wages:
    hoi = i.strip('K').strip('€')
    wage = int(hoi) * 1000
    wages.append(wage)
    
Fifa['Wage'] = wages



In [10]:
list_lengths = np.array(Fifa['Height'])
empty_list = []
for x in list_lengths:
    splitted_values = str(x).split("'")
    feet = (float(x[0])) + (float(x[2])/10)
    centimeters = feet * 30.48
    empty_list.append(round(centimeters))

Fifa['Height'] = empty_list


In [11]:
list_value = Fifa['Value']
list_new_values = []
rows = list_value[:]

for v in rows:
    if v[-1] == 'M':
        strip_value = v.strip('M€')
        if len(strip_value) >= 4:
            yes = (strip_value[:3])
            new_value = float(yes) * 1000000
            list_new_values.append(int(new_value))
        else:
            new_value = float(strip_value) * 1000000
            list_new_values.append(int(new_value))
            
    else:
        strip_value = v.strip('K€')
        new_value = float(strip_value) * 1000
        
        list_new_values.append(int(new_value))

        
Fifa['Value'] = list_new_values

In [12]:
list_weight = Fifa['Weight']
weights = []
for i in list_weight:
    lbs_to_kg = i.strip('lbs')
    convert_weight = float(lbs_to_kg)/2.2046
    roundup_weight = round(convert_weight)
    weights.append(roundup_weight)

Fifa['Weight'] = weights

In [13]:
#onehot encoding
Fifa["Preferred Foot"] = pd.get_dummies(Fifa["Preferred Foot"])

In [14]:
#Rename positions
for i in ['ST', 'CF', 'LF', 'LS', 'LW', 'RF', 'RS', 'RW']:
  Fifa.loc[Fifa.Position == i , 'Position'] = 'Strikers' 

for i in ['CAM', 'CDM', 'LCM', 'CM', 'LAM', 'LDM', 'LM', 'RAM', 'RCM', 'RDM', 'RM']:
  Fifa.loc[Fifa.Position == i , 'Position'] = 'Midfielder' 

for i in ['CB', 'LB', 'LCB', 'LWB', 'RB', 'RCB', 'RWB']:
  Fifa.loc[Fifa.Position == i , 'Position'] = 'Defender' 

In [15]:
#Remove Goalkeepers
Fifa = Fifa.loc[Fifa['Position'] != 'GK', :]

In [16]:
#Checking Dataframe
Fifa.head(5)

,Overall,Potential,Value,Wage,Preferred Foot,Weak Foot,Skill Moves,Work Rate,Body Type,Position,...,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle
Name,,,,,,,,,,,,,,,,,,,,,
L. Messi,94,94,110000000,565000,1,4.0,4.0,Medium/ Medium,Messi,Strikers,...,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0
Cristiano Ronaldo,94,94,77000000,405000,0,4.0,5.0,High/ Low,C. Ronaldo,Strikers,...,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0
Neymar Jr,92,93,118000000,290000,0,5.0,5.0,High/ Medium,Neymar,Strikers,...,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0
K. De Bruyne,91,92,102000000,355000,0,5.0,4.0,High/ High,Normal,Midfielder,...,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0
E. Hazard,91,91,93000000,340000,0,4.0,4.0,High/ Medium,Normal,Strikers,...,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0


In [17]:
#Label encoder for Work Rate and Position
le = preprocessing.LabelEncoder()
Fifa['Work Rate'] = Fifa[['Work Rate']].apply(le.fit_transform)
Fifa['Body Type'] = Fifa[['Body Type']].apply(le.fit_transform)
Fifa['Position'] = Fifa[['Position']].apply(le.fit_transform)

In [42]:
#Checking Dataframe
Fifa.head(5)

,Overall,Potential,Value,Wage,Preferred Foot,Weak Foot,Skill Moves,Work Rate,Body Type,Position,...,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle
Name,,,,,,,,,,,,,,,,,,,,,
L. Messi,94,94,110000000,565000,1,4.0,4.0,8,3,2,...,1.000000,0.510638,0.234043,1.000000,1.000000,75.0,1.021277,0.351064,0.297872,0.276596
Cristiano Ronaldo,94,94,77000000,405000,0,4.0,5.0,1,1,2,...,0.989362,0.670213,0.308511,1.010638,0.872340,85.0,1.010638,0.297872,0.329787,0.244681
Neymar Jr,92,93,118000000,290000,0,5.0,5.0,2,4,2,...,0.891304,0.608696,0.391304,0.967391,0.945652,81.0,1.021739,0.293478,0.260870,0.358696
K. De Bruyne,91,92,102000000,355000,0,5.0,4.0,0,5,1,...,1.000000,0.835165,0.670330,0.956044,1.032967,79.0,0.967033,0.747253,0.637363,0.560440
E. Hazard,91,91,93000000,340000,0,4.0,4.0,2,5,2,...,0.879121,0.593407,0.450549,0.956044,0.978022,86.0,1.000000,0.373626,0.296703,0.241758


In [19]:
FifaSkills = Fifa[['Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle']]

In [20]:
FifaNormSkill = FifaSkills.div(Fifa.Overall, axis=0)
Fifa['Finishing'] = FifaNormSkill['Finishing']
Fifa['HeadingAccuracy'] = FifaNormSkill['HeadingAccuracy']
Fifa['ShortPassing'] = FifaNormSkill['ShortPassing']
Fifa['Volleys'] = FifaNormSkill['Volleys']
Fifa['Dribbling'] = FifaNormSkill['Dribbling']
Fifa['Curve'] = FifaNormSkill['Curve']
Fifa['FKAccuracy'] = FifaNormSkill['FKAccuracy']
Fifa['LongPassing'] = FifaNormSkill['LongPassing']
Fifa['BallControl'] = FifaNormSkill['BallControl']
Fifa['Acceleration'] = FifaNormSkill['Acceleration']
Fifa['SprintSpeed'] = FifaNormSkill['SprintSpeed']
Fifa['Agility'] = FifaNormSkill['Agility']
Fifa['Reactions'] = FifaNormSkill['Reactions']
Fifa['Balance'] = FifaNormSkill['Balance']
Fifa['ShotPower'] = FifaNormSkill['ShotPower']
Fifa['Jumping'] = FifaNormSkill['Jumping']
Fifa['Stamina'] = FifaNormSkill['Stamina']
Fifa['Strength'] = FifaNormSkill['Strength']
Fifa['LongShots'] = FifaNormSkill['LongShots']
Fifa['Aggression'] = FifaNormSkill['Aggression']
Fifa['Interceptions'] = FifaNormSkill['Interceptions']
Fifa['Positioning'] = FifaNormSkill['Positioning']
Fifa['Vision'] = FifaNormSkill['Vision']
Fifa['Composure'] = FifaNormSkill['Composure']
Fifa['Marking'] = FifaNormSkill['Marking']
Fifa['StandingTackle'] = FifaNormSkill['StandingTackle']
Fifa['SlidingTackle'] = FifaNormSkill['SlidingTackle']

In [90]:
#Random Forest
#Features and labels seperation
Features = Fifa.drop(["Position"],axis=1)
Labels = Fifa["Position"]

In [91]:
X = Features
y = Labels

In [92]:
#Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [93]:
#Choose model with hyperparameters
modelrf = RandomForestClassifier(n_estimators=500, max_features=10,max_depth=10)

In [86]:
#Selecting best hyperparameters
param_grid = {
    'n_estimators': [200, 500],
    'max_depth': [10, 50, 100],
    'min_samples_split': [2, 4],
    'max_features': ['sqrt', 'log2']
}

grid = GridSearchCV(modelrf, param_grid = param_grid, cv = 5, verbose = 5, n_jobs = -1)
grid.fit(X_train, y_train)

best_estimator = grid.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  3.1min finished


In [94]:
#fit model
modelrf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features=10, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [99]:
rfval_predict = best_estimator.predict(X_val)

ValueError: Number of features of the model must match the input. Model n_features is 40 and input n_features is 41 

In [ ]:
#Accuracy
print('Accuracy:', metrics.accuracy_score(y_val, rfval_predict))

In [ ]:
#Final Prediction
rf_pred = best_estimator.predict(X_test)
#Final Accuracy
print('Accuracy:', metrics.accuracy_score(y_test, rf_pred))

In [ ]:
#K-NEAREST NEIGHBOURS
#Creating model
modelknn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
#Fit and test model on training/validation set
modelknn.fit(X_train, y_train)
knnval_predict= modelknn.predict(X_val)

In [ ]:
#Validation set accuracy
print("Accuracy:",metrics.accuracy_score(y_val, knnval_predict))

In [ ]:
#Final Prediction
knn_pred = modelknn.predict(X_test)
#Final Accuracy
print('Accuracy:', metrics.accuracy_score(y_test, knn_pred))

In [107]:
#Logistic Regression
modellogreg = LogisticRegression()

In [108]:
#Fit and test model on training/validation set
modellogreg.fit(X_train, y_train)
logregval_predict= modellogreg.predict(X_val)

C:\Users\stane\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\stane\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [109]:
#Validation set accuracy
print("Accuracy:",metrics.accuracy_score(y_val, logregval_predict))

Accuracy: 0.475968992248062


In [110]:
# #Final Prediction
# logregval_predict = modelknn.predict(X_test)
# #Final Accuracy
# print('Accuracy:', metrics.accuracy_score(y_test, logregval_predict))

Accuracy: 0.48527131782945737
